In [301]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## import the required Libraries




In [302]:
import os
import numpy as np
from sklearn.model_selection import train_test_split


from keras.models import Model
from keras.layers import Input, Embedding, SpatialDropout1D, Dropout, Conv1D, MaxPool1D, Flatten, concatenate, Dense, \
    LSTM, Bidirectional, Activation, MaxPooling1D, Add, GRU, GlobalAveragePooling1D, GlobalMaxPooling1D, RepeatVector, \
    TimeDistributed, Permute, multiply, Lambda, add, Masking, BatchNormalization, Softmax, Reshape, ReLU, \
    ZeroPadding1D, subtract
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import tensorflow.keras.backend as K
import tensorflow as tf
from keras import backend as K, initializers, regularizers, constraints


# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from tensorflow.keras.layers import Layer, InputSpec
import numpy as np

from statistics import mode

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
%matplotlib inline
import tensorflow as tf


# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense, GlobalAveragePooling1D
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.metrics import Metric
from tensorflow.python.keras.utils import metrics_utils
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.keras.utils.generic_utils import to_list
import tensorflow_hub as hub



import re
import nltk
from nltk.corpus import stopwords

# neural
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential

#Metrics
from sklearn.metrics import balanced_accuracy_score

from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.plotting import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier

##LOAD THE SEMEVAL LAPTOP DATASET


In [303]:
df_train = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_SemEval2016/laptops_train_single.csv", encoding='utf8')

In [304]:
df2_train, df2_val = train_test_split(df_train, test_size=0.2, random_state=42)


In [305]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_SemEval2016/laptops_test_single.csv", encoding='utf8')

In [306]:
#df2_val = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_Sentihood/sentihood_dev_single.csv", encoding='utf8')

##PREPROCESSING

In [307]:
df2_test['aspect_category'] = df2_test['aspect_category'].str.replace('#', ' ')
df2_train['aspect_category'] = df2_train['aspect_category'].str.replace('#', ' ')
df2_val['aspect_category'] = df2_val['aspect_category'].str.replace('#', ' ')

In [308]:
## change the labels to integers for the training phase
df2_train['polarity'][df2_train["polarity"] == 'negative'] = 0
df2_train['polarity'][df2_train["polarity"] == 'positive'] = 1
df2_train['polarity'][df2_train["polarity"] == 'neutral'] = 2



In [309]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1
df2_test['polarity'][df2_test["polarity"] == 'neutral'] = 2


In [310]:
df2_val['polarity'][df2_val["polarity"] == 'negative'] = 0
df2_val['polarity'][df2_val["polarity"] == 'positive'] = 1
df2_val['polarity'][df2_val["polarity"] == 'neutral'] = 2

In [311]:
train_df = df2_train

In [312]:
## Convert polarity Values to Integers
train_df["polarity"] = pd.to_numeric(train_df["polarity"],errors='coerce')


In [313]:
## Split the Dataset into Training and Testing Datasets
#X_train, y_train= train_test_split(train_df,train_df['polarity'],stratify=train_df['polarity'])

X_train  = train_df
y_train = train_df['polarity']

In [314]:
X_test = df2_test

In [315]:
## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [316]:
y_test = X_test["polarity"]

In [317]:
X_val = df2_val

In [318]:
## Convert polarity Values to Integers
X_val["polarity"] = pd.to_numeric(X_val["polarity"],errors='coerce')


In [319]:
y_val = X_val["polarity"]

In [320]:
NUM_WORDS = 100000 ## MAx of words to keep, based on word frequency.
EMBEDDING_SIZE = 128 ## the length of the Vector the will

## Embedding code


In [321]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_train_tokenized = tokenizer.texts_to_sequences(X_train.sentence) # list of tokenized sentences
Aspect_X_train_tokenized = tokenizer.texts_to_sequences(X_train['aspect_category']) # list of tokenized sentences
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_category']) # list of tokenized sentences
text_X_val_tokenized = tokenizer.texts_to_sequences(X_val.sentence) # list of tokenized sentences
Aspect_X_val_tokenized = tokenizer.texts_to_sequences(X_val['aspect_category']) # list of tokenized sentences

In [322]:
##Maximum length of sentences
Max_Len =  max([len(one_title) for one_title in text_X_train_tokenized])
Max_Len

73

In [323]:
## Pads sequences(or sentences) to the same length.


text_X_train_padded = pad_sequences(text_X_train_tokenized, maxlen=Max_Len)
aspect_X_train_padded = pad_sequences(Aspect_X_train_tokenized, maxlen=1)

text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)

text_X_val_padded = pad_sequences(text_X_val_tokenized, maxlen=Max_Len)
aspect_X_val_padded = pad_sequences(Aspect_X_val_tokenized, maxlen=1)


In [324]:
## Concatenate the Embedding presentation of the sentences and aspects to Create training and testing dataset
train_data =  [text_X_train_padded,aspect_X_train_padded]
test_data =   [text_X_test_padded,aspect_X_test_padded]
val_data = [text_X_val_padded,aspect_X_val_padded]

In [325]:

# modified based on `https://gist.github.com/cbaziotis/7ef97ccf71cbc14366835198c09809d2`
class Attention(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
 e: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self, W_regularizer=None, u_regularizer=None, b_regularizer=None, W_constraint=None,
                 u_constraint=None, b_constraint=None, use_W=True, use_bias=False, return_self_attend=False,
                 return_attend_weight=True, **kwargs):
        self.supports_masking = True

        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.use_W = use_W
        self.use_bias = use_bias
        self.return_self_attend = return_self_attend    # whether perform self attention and return it
        self.return_attend_weight = return_attend_weight    # whether return attention weight
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        if self.use_W:
            self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),  initializer=self.init,
                                     name='{}_W'.format(self.name), regularizer=self.W_regularizer,
                                     constraint=self.W_constraint)
        if self.use_bias:
            self.b = self.add_weight(shape=(input_shape[1],), initializer='zero', name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer, constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],), initializer=self.init, name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer, constraint=self.u_constraint)

        super(Attention, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        if self.use_W:
            x = K.tanh(K.dot(x, self.W))

        ait = Attention.dot_product(x, self.u)
        if self.use_bias:
            ait += self.b

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        if self.return_self_attend:
            attend_output = K.sum(x * K.expand_dims(a), axis=1)
            if self.return_attend_weight:
                return [attend_output, a]
            else:
                return attend_output
        else:
            return a

    def compute_output_shape(self, input_shape):
        if self.return_self_attend:
            if self.return_attend_weight:
                return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[1])]
            else:
                return input_shape[0], input_shape[-1]
        else:
            return input_shape[0], input_shape[1]

    @staticmethod
    def dot_product(x, kernel):
        """
        Wrapper for dot product operation, in order to be compatible with both
        Theano and Tensorflow
        Args:
            x (): input
            kernel (): weights
        Returns:
        """
        if K.backend() == 'tensorflow':
            return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
        else:
            return K.dot(x, kernel)

In [326]:
# attention-based lstm with aspect embedding
def atae_lstm(lstm_units = 128):
    input_text = Input(shape=(Max_Len,))
    input_aspect = Input(shape=(1,),)

    word_embedding = Embedding(NUM_WORDS, EMBEDDING_SIZE, input_length=Max_Len)
    text_embed = SpatialDropout1D(0.2)(word_embedding(input_text))

    asp_embedding = Embedding(NUM_WORDS, EMBEDDING_SIZE, input_length=Max_Len)

    aspect_embed = asp_embedding(input_aspect)
    aspect_embed = Flatten()(aspect_embed)  # reshape to 2d
    repeat_aspect = RepeatVector(Max_Len)(aspect_embed)  # repeat aspect for every word in sequence

    input_concat = concatenate([text_embed, repeat_aspect], axis=-1)
    print(input_concat.shape)
    hidden_vecs, state_h, _ = LSTM(lstm_units, return_sequences=True, return_state=True)(input_concat)
    concat = concatenate([hidden_vecs, repeat_aspect], axis=-1)

    # apply attention mechanism
    attend_weight = Attention()(concat)
    attend_weight_expand = Lambda(lambda x: K.expand_dims(x))(attend_weight)
    attend_hidden = multiply([hidden_vecs, attend_weight_expand])
    attend_hidden = Lambda(lambda x: K.sum(x, axis=1))(attend_hidden)

    attend_hidden_dense = Dense(lstm_units)(attend_hidden)
    last_hidden_dense = Dense(lstm_units)(state_h)
    final_output = Activation('tanh')(add([attend_hidden_dense, last_hidden_dense]))
    output = Dense(3, activation='softmax')(final_output)
    return Model([input_text, input_aspect], output)

In [327]:
atae_lstm_model =  atae_lstm()
atae_lstm_model.summary()

(None, 73, 256)
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, 73)]         0           []                               
                                                                                                  
 embedding_11 (Embedding)       (None, 1, 128)       12800000    ['input_12[0][0]']               
                                                                                                  
 embedding_10 (Embedding)       (None, 73, 128)      12800000    ['input_11[0][0]']               
                                                                            

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [328]:

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                                    patience = 2,
                                                    verbose=1,
                                                    factor=0.1,
                                                    min_lr=0.000001)

atae_lstm_model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
atae_lstm_model.fit(x = train_data, y = y_train, validation_data = (val_data, y_val), batch_size=16, epochs=10, callbacks = [learning_rate_reduction])

Epoch 1/10
146/146 [==============================] - 17s 90ms/step - loss: 0.7073 - accuracy: 0.6798 - val_loss: 0.6980 - val_accuracy: 0.7474 - lr: 0.0010
Epoch 2/10
146/146 [==============================] - 5s 33ms/step - loss: 0.4440 - accuracy: 0.8328 - val_loss: 0.6088 - val_accuracy: 0.7749 - lr: 0.0010
Epoch 3/10
146/146 [==============================] - 5s 31ms/step - loss: 0.2904 - accuracy: 0.8956 - val_loss: 0.6007 - val_accuracy: 0.7869 - lr: 0.0010
Epoch 4/10
146/146 [==============================] - 3s 21ms/step - loss: 0.2080 - accuracy: 0.9248 - val_loss: 0.7221 - val_accuracy: 0.7869 - lr: 0.0010
Epoch 5/10
146/146 [==============================] - ETA: 0s - loss: 0.1680 - accuracy: 0.9373
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
146/146 [==============================] - 3s 19ms/step - loss: 0.1680 - accuracy: 0.9373 - val_loss: 0.7414 - val_accuracy: 0.7852 - lr: 0.0010
Epoch 6/10
146/146 [==============================] - 3s 

In [329]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
26/26 [==============================] - 0s 5ms/step - loss: 1.2799 - accuracy: 0.6829
test loss, test acc: [1.279923439025879, 0.6828963756561279]
Accuracy of the model is -  68.2896375656128 %


##CROSS VALIDATION

##CROSS VALIDATE ON SENTIHOOD

In [330]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_Sentihood/sentihood_test_single.csv", encoding='utf8')

In [331]:
df2_test.drop(columns=['to' , 'from'], inplace=True)

In [332]:
## change the labels to integers for the training phase
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1



<ipython-input-332-d7e862dbc869>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
<ipython-input-332-d7e862dbc869>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1


In [333]:
X_test = df2_test

In [334]:
## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [335]:
y_test = X_test["polarity"]

In [336]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_category']) # list of tokenized sentences


In [337]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [338]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [339]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
53/53 [==============================] - 0s 5ms/step - loss: 1.6138 - accuracy: 0.5878
test loss, test acc: [1.613752841949463, 0.5878499150276184]
Accuracy of the model is -  58.78499150276184 %


##CROSS VALIDATE ON SEMEVAL Restaurant

In [342]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_SemEval2016/restaurants_test_single.csv", encoding='utf8')

In [346]:
df2_test.drop(columns=['from' , 'to', 'aspect_term'], inplace=True)



In [347]:
df2_test

,sentence Id,sentence,aspect_category,polarity
0,en_BlueRibbonSushi_478218171:0,Yum!,FOOD QUALITY,positive
1,en_BlueRibbonSushi_478218171:1,Serves really good sushi.,FOOD QUALITY,positive
2,en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,FOOD STYLE_OPTIONS,neutral
3,en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,FOOD QUALITY,positive
4,en_BlueRibbonSushi_478218171:4,Don't leave the restaurant without it.,FOOD QUALITY,positive
...,...,...,...,...
854,en_Ray'sBoathouse_477775918:13,"All considered, I have to say that Ray's Boath...",RESTAURANT GENERAL,positive
855,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,SERVICE GENERAL,positive
856,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,FOOD QUALITY,positive
857,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,RESTAURANT MISCELLANEOUS,negative


In [348]:
df2_test['aspect_category'] = df2_test['aspect_category'].str.replace('#', ' ')

In [349]:
df2_test

,sentence Id,sentence,aspect_category,polarity
0,en_BlueRibbonSushi_478218171:0,Yum!,FOOD QUALITY,positive
1,en_BlueRibbonSushi_478218171:1,Serves really good sushi.,FOOD QUALITY,positive
2,en_BlueRibbonSushi_478218171:2,Not the biggest portions but adequate.,FOOD STYLE_OPTIONS,neutral
3,en_BlueRibbonSushi_478218171:3,Green Tea creme brulee is a must!,FOOD QUALITY,positive
4,en_BlueRibbonSushi_478218171:4,Don't leave the restaurant without it.,FOOD QUALITY,positive
...,...,...,...,...
854,en_Ray'sBoathouse_477775918:13,"All considered, I have to say that Ray's Boath...",RESTAURANT GENERAL,positive
855,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,SERVICE GENERAL,positive
856,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,FOOD QUALITY,positive
857,en_Ray'sBoathouse_477775918:14,While I could have done without the youth who ...,RESTAURANT MISCELLANEOUS,negative


In [350]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1
df2_test['polarity'][df2_test["polarity"] == 'neutral'] = 2


In [351]:
X_test = df2_test

In [352]:
## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [353]:
y_test = X_test["polarity"]

In [354]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_category']) # list of tokenized sentences


In [355]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [356]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [357]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
27/27 [==============================] - 0s 5ms/step - loss: 1.2919 - accuracy: 0.6764
test loss, test acc: [1.2918760776519775, 0.6763678789138794]
Accuracy of the model is -  67.63678789138794 %


##CROSS VALIDATE ON MAMS

In [358]:
df2_test = pd.read_csv("/content/drive/MyDrive/ABSA work/codes/processed_MAMS/ACSA_test_single.csv", encoding='utf8')

In [359]:
df2_test['polarity'][df2_test["polarity"] == 'negative'] = 0
df2_test['polarity'][df2_test["polarity"] == 'positive'] = 1
df2_test['polarity'][df2_test["polarity"] == 'neutral'] = 2

In [360]:
X_test = df2_test

In [361]:
## Convert polarity Values to Integers
X_test["polarity"] = pd.to_numeric(X_test["polarity"],errors='coerce')


In [362]:
y_test = X_test["polarity"]

In [363]:
tokenizer = Tokenizer(num_words=NUM_WORDS,
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, )
## Fit_on_texts : Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(list(X_train.sentence))
text_X_test_tokenized = tokenizer.texts_to_sequences(X_test.sentence) # list of tokenized sentences
Aspect_X_test_tokenized = tokenizer.texts_to_sequences(X_test['aspect_category']) # list of tokenized sentences


In [364]:
## Pads sequences(or sentences) to the same length.



text_X_test_padded = pad_sequences(text_X_test_tokenized, maxlen=Max_Len)
aspect_X_test_padded = pad_sequences(Aspect_X_test_tokenized, maxlen=1)


In [365]:
test_data =   [text_X_test_padded,aspect_X_test_padded]


In [366]:
print("Evaluate on test data")
results = atae_lstm_model.evaluate(x = test_data , y = y_test )
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")

Evaluate on test data
29/29 [==============================] - 0s 7ms/step - loss: 3.5275 - accuracy: 0.3219
test loss, test acc: [3.527467966079712, 0.32186460494995117]
Accuracy of the model is -  32.18646049499512 %
